In [1]:
import pandas as pd
import numpy as np
from os import listdir

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
def HeaderMaker(PatientFile):
    df_cleaned = pd.read_csv('/data/ortho/' + PatientFile, header=None)

    df_cleaned = df_cleaned.rename(columns={0: "thorax_r_x", 1: "thorax_r_y", 2: "thorax_r_z"})
    df_cleaned = df_cleaned.rename(columns={3: "clavicula_r_x", 4: "clavicula_r_y", 5: "clavicula_r_z"})
    df_cleaned = df_cleaned.rename(columns={6: "scapula_r_x", 7: "scapula_r_y", 8: "scapula_r_z"})
    df_cleaned = df_cleaned.rename(columns={9: "humerus_r_x", 10: "humerus_r_y", 11: "humerus_r_z"})
    df_cleaned = df_cleaned.rename(columns={12: "ellebooghoek_r"})
    df_cleaned = df_cleaned.rename(columns={15: "thorax_l_x", 16: "thorax_l_y", 17: "thorax_l_z"})
    df_cleaned = df_cleaned.rename(columns={18: "clavicula_l_x", 19: "clavicula_l_y", 20: "clavicula_l_z"})
    df_cleaned = df_cleaned.rename(columns={21: "scapula_l_x", 22: "scapula_l_y", 23: "scapula_l_z"})
    df_cleaned = df_cleaned.rename(columns={24: "humerus_l_x", 25: "humerus_l_y", 26: "humerus_l_z"})
    df_cleaned = df_cleaned.rename(columns={27: "ellebooghoek_l"})
    
    return df_cleaned

In [3]:

Patients = {}
testdataurl = '/data/ortho/'
for PatientFile in listdir(testdataurl):
    if ('meting' not in PatientFile):
        print('Skipping: %s' % PatientFile)
        # Skip this iteration
        continue
    Splitted = PatientFile.split('_')
    Patient = Splitted[0] + '_' + Splitted[1]

    Data = HeaderMaker(PatientFile)
    MaxDict = Data.max().to_dict() # .abs() for absolute numbers
    
    if(Patient not in Patients.keys()):
        Patients[Patient] = MaxDict
        Patients[Patient]['GroundTruth'] = Splitted[0].split('Cat')[1]
        Patients[Patient]['Bias'] = 0
        continue
    
    for key in MaxDict.keys():
        Patients[Patient][key]= max(Patients[Patient][key],MaxDict[key])
        

    
    
            
            
            
            

Skipping: StudentNummers
Skipping: testset
Skipping: AllPatients.csv
Skipping: TestDataAllPatients.csv


In [4]:
df = pd.DataFrame.from_dict(Patients, orient='index')
print(df.columns.values)

['thorax_r_x' 'thorax_r_y' 'thorax_r_z' 'clavicula_r_x' 'clavicula_r_y'
 'clavicula_r_z' 'scapula_r_x' 'scapula_r_y' 'scapula_r_z' 'humerus_r_x'
 'humerus_r_y' 'humerus_r_z' 'ellebooghoek_r' 13 14 'thorax_l_x'
 'thorax_l_y' 'thorax_l_z' 'clavicula_l_x' 'clavicula_l_y' 'clavicula_l_z'
 'scapula_l_x' 'scapula_l_y' 'scapula_l_z' 'humerus_l_x' 'humerus_l_y'
 'humerus_l_z' 'ellebooghoek_l' 28 29 'GroundTruth' 'Bias']


In [5]:
Xcolumns = ['Bias']
Xcolumns.extend(['thorax_r_x', 'thorax_r_y', 'thorax_r_z', 'clavicula_r_x', 'clavicula_r_y',
 'clavicula_r_z', 'scapula_r_x', 'scapula_r_y', 'scapula_r_z', 'humerus_r_x',
 'humerus_r_y', 'humerus_r_z', 'ellebooghoek_r', 'thorax_l_x',
 'thorax_l_y', 'thorax_l_z', 'clavicula_l_x', 'clavicula_l_y', 'clavicula_l_z',
 'scapula_l_x', 'scapula_l_y', 'scapula_l_z', 'humerus_l_x', 'humerus_l_y',
 'humerus_l_z', 'ellebooghoek_l'])

X = df[Xcolumns]
y = df[['GroundTruth']]


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [7]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [8]:
y_pred = lr.predict(X_test)
print(lr.coef_[0])
print(metrics.accuracy_score(y_test, y_pred))

[ 0.         -0.21529365 -0.40986032 -0.21249066  0.15423425  0.28928863
  0.07695528  0.33868736 -0.13525988 -0.07461502 -0.1699128   0.48100461
  0.04322715 -0.00131356 -0.21529365 -0.07919595 -0.54222049 -0.18374553
 -0.28971006  0.01243582 -0.41306239 -0.51043754 -0.25889919 -0.42158465
 -0.01644535  0.05917898 -0.00062386]
0.7142857142857143


In [9]:
len(Patients['Cat3_pat38'])

32